In [1]:
import sys
sys.path.append('..')

import numpy as np
import socnet as sn

In [2]:
sn.graph_width = 500
sn.graph_height = 500
sn.node_size = 10
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

In [3]:
total_nodes = 500
percentage_portuguese = 0.05
percentage_native = 1 - percentage_portuguese

g = sn.generate_empty_graph(total_nodes)

In [4]:
from random import randint


initial_population = {
    'portuguese': 0,
    'native': 0
}

for n in range(int(total_nodes * percentage_portuguese)):
    g.node[n]['nationality'] = 'portuguese'
    g.node[n]['color'] = (255,255,0)
    g.node[n]['label'] = n
    pop = randint(20, 50)
    g.node[n]['population'] = pop
    initial_population['portuguese'] += pop
    
for n in range(int(total_nodes * percentage_portuguese),
               total_nodes):
    g.node[n]['nationality'] = 'native'
    g.node[n]['color'] = (255,0,255)
    g.node[n]['label'] = n
    pop = randint(40, 200)
    g.node[n]['population'] = pop
    initial_population['native'] += pop


In [5]:
from random import randint, choice

N = g.number_of_nodes() // 5  # each node should have aprox. 5 connections

for n in g.nodes():
    for i in g.nodes():
        if n != i and randint(0, N) == 0:
            g.add_edge(n,i)
            g.edges[(n, i)]['label'] = choice(['+', '-'])

In [6]:
sn.reset_edge_colors(g)
sn.reset_positions(g)
sn.show_graph(g, nlab=True)

In [7]:
from random import choice
from functools import reduce


def _calc_pt(g, n):
    return sum(list(map(lambda x: x[1], 
                   filter(lambda x: x[0]['nationality'] == 'portuguese', 
                        map(lambda x: (g.node[x], 1), list(g.neighbors(n)))))))


def fight_native_vs_native(g, n0, n1):
    '''
    returns the looser of a battle between natives
    takes into consideration having portuguese neighbors
    as an advantage
    '''
    pt0 = _calc_pt(g, n0)
    pt1 = _calc_pt(g, n1)
    if pt0 == pt1:
        return n0 if choice([pt0, pt1]) == pt0 else n1
    elif pt0 > pt1:
        return n1
    return n0

In [8]:
from random import choice, randint


def portuguese_vs_native(g, n0, n1):
    '''
    returns true if the first one is pt 
    and the second one is a native 
    '''
    return (g.node[n0]['nationality'] == 'portuguese' and \
            g.node[n1]['nationality'] == 'native')


def fight(g, n0, n1):
    '''
    returns the looser of a simulated fight
    '''    
    P = 70  # P * 10% chance a portuguese will win
    if g.node[n0]['nationality'] == g.node[n1]['nationality'] and g.node[n0]['nationality'] == 'native':
        return fight_native_vs_native(g, n0, n1)
    if g.node[n0]['nationality'] == g.node[n1]['nationality'] and g.node[n0]['nationality'] == 'portuguese':
        return choice([n0, n1])
    elif portuguese_vs_native(g, n1, n0):
        return n1 if randint(1, 100) > P else n0
    elif portuguese_vs_native(g, n0, n1):
        return n0 if randint(1, 100) > P else n1
    else:
        return -1  # error

In [9]:
from random import randint


def _remove_lone_nodes(g):
    remove = set()
    for n in g.nodes():
        if len(list(g.neighbors(n))) == 0:
            remove.add(n)
            
    g.remove_nodes_from(remove)

    
def flip_edge(g, m, n):
    e = g.edges[n, m]['label']
    g.edges[n, m]['label'] = '-' if e == '+' else '+'
    

def resolve_combat(g, fighters):
    for l, w in fighters:
        # winner split population, looser dies
        pop = g.node[w]['population'] // 2  # integer division, no such a thing as half person
        g.node[w]['population'] -= pop
        g.node[l]['population'] = pop
        g.node[l]['nationality'] = g.node[w]['nationality']  # conversion
        
        for n in list(g.neighbors(l)):
            flip_edge(g, l, n)
        


def iteration(g):
    # TODO: Implement population growth
    P = 1  # 10% chance a '-' edge will fight
    fighters = set()
    for n, m in g.edges():
        if g.edges[n, m]['label'] == '-' and randint(1, 10) <= P:
            l = fight(g, n, m)
            fighters.add((l, m if l == n else n))  # (looser, winner)
    
    resolve_combat(g, fighters)
#     g.remove_nodes_from(remove)
#     _remove_lone_nodes(g)

In [10]:
def snapshot(g, frames):
    frame = sn.generate_frame(g)
    frames.append(frame)

In [11]:
frames = []

for _ in range(5):
    iteration(g)
#     sn.reset_positions(g)
    snapshot(g, frames)
    
# sn.show_animation(frames)

In [12]:
pt = 0
nt = 0

for n in g.nodes():
    if g.node[n]['nationality'] == 'portuguese':
        pt += g.node[n]['population']
#         pt += 1
    else:
        nt += g.node[n]['population']
#         nt += 1
    
print('''
    portuguese: {:6d} -- {:3.2f}% of total -- {:3.2f}% of initial
    native:     {:6d} -- {:3.2f}% of total -- {:3.2f}% of initial
'''.format(pt, (pt / (pt + nt)) * 100, pt / initial_population['portuguese'] * 100,
           nt, (nt / (pt + nt)) * 100, nt / initial_population['native'] * 100))


    portuguese:    447 -- 2.81% of total -- 53.66% of initial
    native:      15479 -- 97.19% of total -- 26.86% of initial

